In [49]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os
from datetime import datetime
import time
import logging
import dotenv

In [55]:
df = pd.read_csv("emails.csv")
df.head()

,EMAILS
0,info@cargen.com
1,elizabeth@charteredengineering.com
2,info@firstlenders.co.ke
3,amina@blackwoodhodge.com
4,sales@hydromaticsea.com


In [56]:
df.isna().sum()
display(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   EMAILS   185 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


None

In [52]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='bulk-email-sender.log',
)

In [53]:
class BulkEmailSender:
    def __init__(self, smtp_server, smtp_port, username, password, sender_email, sender_name=None):        
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.username = username
        self.password = password
        self.sender_email = sender_email
        self.sender_name = sender_name
        self.session = None
    
    def connect(self):
        try:
            self.session = smtplib.SMTP(self.smtp_server, self.smtp_port)
            self.session.ehlo()
            self.session.starttls() # enable encryption
            self.session.ehlo()
            self.session.login(self.username, self.password)
            logging.info("Successfully connected to SMTP server")
        except Exception as e:
            logging.error(f"Failed to connect to SMTP server: {e}")
            return False
        return True
    
    def disconnect(self):
        if self.session:
            self.session.quit()
            self.session = None
            logging.info("Disconnected from SMTP server")
    
    def create_message(self, recipient_email, subject, html_content, text_content=None, attachment_paths=None, cc=None, bcc=None):
        msg = MIMEMultipart("alternative")
        msg["subject"] = subject
        msg["From"] = f"{self.sender_name} <{self.sender_email}>"
        msg["To"] = recipient_email
        
        if cc:
            msg["Cc"] = ", ".join(cc)
        if bcc:
            msg["Bcc"] = ", ".join(bcc)
        
        if text_content:
            msg.attach(MIMEText(text_content, "plain"))
        msg.attach(MIMEText(html_content, "html"))
        
        if attachment_paths:
            for file_path in attachment_paths:
                if os.path.exists(file_path):
                    with open(file_path, "rb") as f:
                        attachment = MIMEApplication(f.read(), Name=os.path.basename(file_path))
                        attachment["Content-Disposition"] = f"attachment; filename={os.path.basename(file_path)}"
                        msg.attach(attachment)
                else:
                    logging.warning(f"Attachment not found: {file_path}")
        
        return msg
    
    def send_email(self, msg, recipient_email, cc=None, bcc=None):
        if not self.session:
            if not self.connect():
                return False
        
        recipients = [recipient_email]
        if cc:
            recipients.extend(cc)
        if bcc:
            recipients.extend(bcc)
        
        try:
            self.session.send_message(msg, self.sender_email, recipients)
            logging.info(f"Email sent to {recipient_email}")
            return True
        except Exception as e:
            logging.error(f"Failed to send email to {recipient_email}: {e}")
            return False
    
    def send_bulk_emails(self, csv_file, subject, html_template, text_template=None, attachment_paths=None, personalize=True, delay_base=1, max_emails_per_day=450):
        if not self.session:
            if not self.connect():
                return False
        results = {"success": 0, "failed": 0, "skipped": 0}
        try:
            df = pd.read_csv(csv_file)    
            
            if "Emails" not in df.columns:
                logging.error("Invalid csv file, must contain an `Emails` column")
            
            df = df.dropna(subset=["Emails"])
            
            if len(df) > max_emails_per_day:
                logging.warning(f"Limiting to {max_emails_per_day} emails per day, due to Gmail's daily sending limit")
                df = df.head(max_emails_per_day)
            
            total_recipients = len(df)
            start_time = datetime.now()
            logging.info(f"Starting bulk email campaign to {total_recipients} recipients")
            
            for index, row in df.iterrows():
                recipient_email = row["Emails"].strip()
                
                # Personalize content if needed
                if personalize:
                    personalized_html = html_template
                    personalized_text = text_template if text_template else None
                    
                    # Replace placeholders with recipient data
                    for key, value in row.items():
                        placeholder = f"{{{{{key}}}}}"
                        # Handle NaN values
                        str_value = "" if pd.isna(value) else str(value)
                        personalized_html = personalized_html.replace(placeholder, str_value)
                        if personalized_text:
                            personalized_text = personalized_text.replace(placeholder, str_value)
                else:
                    personalized_html = html_template
                    personalized_text = text_template
                
                # Process CC and BCC if they exist in the dataframe
                cc = None
                if 'cc' in row and not pd.isna(row['cc']):
                    cc = [email.strip() for email in str(row['cc']).split(',')] if ',' in str(row['cc']) else row['cc']
                
                bcc = None
                if 'bcc' in row and not pd.isna(row['bcc']):
                    bcc = [email.strip() for email in str(row['bcc']).split(',')] if ',' in str(row['bcc']) else row['bcc']
                
                
                  # Create and send the email
                msg = self.create_message(
                    recipient_email=recipient_email,
                    subject=subject,
                    html_content=personalized_html,
                    text_content=personalized_text,
                    attachment_paths=attachment_paths,
                    cc=cc,
                    bcc=bcc
                )  
                
                if self.send_email(msg, recipient_email, cc, bcc):
                    results["success"] += 1
                else:
                    results["failed"] += 1
                
                # Log progress for large batches
                if (index + 1) % 10 == 0:
                    logging.info(f"Progress: {index + 1}/{total_recipients} emails sent")
                
                # Add 0.5 secondd for every 50 emails
                current_delay = delay_base + (index // 50) * 0.5
                if current_delay > 6:  # Cap at 10 seconds
                    current_delay = 6
                    
                if index < total_recipients - 1:
                    time.sleep(current_delay)
                
                end_time = datetime.now()
                duration = (end_time - start_time).total_seconds()
                logging.info(f"Bulk email campaign completed in {duration:.2f} seconds")
                logging.info(f"Results: {results['success']} successful, {results['failed']} failed, {results['skipped']} skipped")
            
        except Exception as e:
            logging.error(f"Error in buld email process: {str(e)}")
            return False
        finally:
            self.disconnect()
        
        return results


In [54]:
if __name__ == "__main__":
    # Load environment variables from .env file
    dotenv.load_dotenv()
    
    # HTML email template with placeholders
    html_template = """
    <!DOCTYPE html>
    <html>
    <body>
        <h2>Hello {{name}},</h2>
        <p>Thank you for your interest in our products at {{company}}.</p>
        <p>Please find our latest catalog attached.</p>
        <p>Best regards,<br>Sales Team</p>
        <p style="font-size: 12px; color: gray;">
            If you wish to unsubscribe from future communications, 
            <a href="https://example.com/unsubscribe?email={{email}}">click here</a>.
        </p>
    </body>
    </html>
    """
    
    # Plain text alternative
    text_template = """
    Hello {{name}},
    
    Thank you for your interest in our products at {{company}}.
    Please find our latest catalog attached.
    
    Best regards,
    Sales Team
    
    To unsubscribe, visit: https://example.com/unsubscribe?email={{email}}
    """
    
    # Initialize the sender using environment variables
    sender = BulkEmailSender(
        smtp_server='smtp.gmail.com',
        smtp_port=587,
        username=os.environ.get('EMAIL_USERNAME'),
        password=os.environ.get('EMAIL_PASSWORD'),
        sender_email=os.environ.get('SENDER_EMAIL'),
        sender_name='Company Sales Team'
    )
    
    # Send bulk emails using pandas
    results = sender.send_bulk_emails(
        csv_file='testing.csv',
        subject='New Product Catalog',
        html_template=html_template,
        text_template=text_template,
        attachment_paths=["sample_cv.pdf"],
        personalize=True,
        delay_base=2,  # Base delay between emails
        max_emails_per_day=450  # Stay under Gmail's limit with some buffer
    )
    
    print(f"Email campaign summary: {results}")

Email campaign summary: {'success': 2, 'failed': 0, 'skipped': 0}
